# 1. Collecting historical data (12 marks)
Using the 2-year historical daily stock prices from the attached spreadsheet (from 31/12/2020 to 30/12/2022), compute the parameters for each stock and stock pairs, which would be used in the next part of this assignment. [Formulas as in Lecture Powerpoint Topic 1-1, slide 97 or Course Notes Chapter 2, p.33]

### Import the libs and the data

In [1]:
import pandas as pd
import numpy as np
# 从CSV文件中读取股票价格数据
stock_prices = pd.read_csv('./data/stock_prices.csv')
stock_prices.head()

,Date,1,5,16,288,388,823,883,2020,2269,2628
0,2020/12/31,54.50,40.65,100.1,6.53,430.0,70.85,7.22,123.0,102.00,17.12
1,2021/1/4,55.65,40.45,103.3,6.72,441.8,71.75,7.05,127.0,103.20,17.10
2,2021/1/5,56.00,40.15,106.6,7.17,457.0,73.10,7.06,129.4,101.90,16.90
3,2021/1/6,56.10,41.50,106.6,7.06,449.0,72.75,7.36,132.8,98.00,17.10
4,2021/1/7,57.00,43.40,106.6,6.89,443.0,71.45,7.25,130.7,94.95,17.10


## 1.1 Annualized standard deviation of daily return.

In [2]:
# 计算每日收益率
daily_returns = stock_prices.drop(columns=['Date']).pct_change().drop(index=0)
# 计算日收益率的标准差
daily_std = np.std(daily_returns)
# 将日标准差乘以根号252得到年化标准差
annualized_std = daily_std * np.sqrt(252)
annualized_std

1       0.222806
5       0.270339
16      0.219044
288     0.344072
388     0.374550
823     0.205795
883     0.341429
2020    0.503378
2269    0.716768
2628    0.279444
dtype: float64

## 1.2 Annualized covariance between each pair of stocks.

In [3]:
# 使用numpy的cov函数计算协方差矩阵
covariance_matrix = np.cov(daily_returns, rowvar=False)
# 将协方差矩阵乘以252得到年化协方差矩阵（老师要求乘以252）
annualized_covariance_matrix = covariance_matrix * 252
annualized_covariance_matrix

array([[0.04974327, 0.03095847, 0.02365935, 0.02953636, 0.02714655,
        0.02074297, 0.02768582, 0.03040187, 0.02263696, 0.03133031],
       [0.03095847, 0.07323199, 0.0228838 , 0.02668437, 0.02892804,
        0.01934454, 0.03282888, 0.02430854, 0.01577333, 0.03525235],
       [0.02365935, 0.0228838 , 0.04807796, 0.02679659, 0.0277105 ,
        0.02472931, 0.01780029, 0.02908567, 0.02954222, 0.0226202 ],
       [0.02953636, 0.02668437, 0.02679659, 0.11862591, 0.0457027 ,
        0.02250091, 0.02892033, 0.04985813, 0.05283297, 0.03825967],
       [0.02714655, 0.02892804, 0.0277105 , 0.0457027 , 0.14057282,
        0.02772218, 0.03592605, 0.11335731, 0.14323618, 0.05911404],
       [0.02074297, 0.01934454, 0.02472931, 0.02250091, 0.02772218,
        0.04243761, 0.01594923, 0.02395307, 0.0311021 , 0.02283678],
       [0.02768582, 0.03282888, 0.01780029, 0.02892033, 0.03592605,
        0.01594923, 0.11681094, 0.04071138, 0.04611276, 0.04016451],
       [0.03040187, 0.02430854, 0.0290856

## 1.3 Correlation coefficient between each pair of stocks.

In [4]:
# 计算协方差矩阵的相关系数矩阵
correlation_matrix = np.corrcoef(daily_returns, rowvar=False)
correlation_matrix

array([[1.        , 0.51293455, 0.48379641, 0.38450329, 0.3246361 ,
        0.45146927, 0.36320222, 0.27051885, 0.14145928, 0.50218312],
       [0.51293455, 1.        , 0.38566009, 0.28629699, 0.28511354,
        0.34700249, 0.35494726, 0.17826783, 0.0812369 , 0.46569569],
       [0.48379641, 0.38566009, 1.        , 0.35482701, 0.33707044,
        0.54747405, 0.2375266 , 0.26325134, 0.18778057, 0.36879757],
       [0.38450329, 0.28629699, 0.35482701, 1.        , 0.35391696,
        0.31712815, 0.24568103, 0.28728351, 0.21379411, 0.39711477],
       [0.3246361 , 0.28511354, 0.33707044, 0.35391696, 1.        ,
        0.35892292, 0.28036059, 0.60001667, 0.53245454, 0.56364359],
       [0.45146927, 0.34700249, 0.54747405, 0.31712815, 0.35892292,
        1.        , 0.22652837, 0.23075454, 0.21042377, 0.39629995],
       [0.36320222, 0.35494726, 0.2375266 , 0.24568103, 0.28036059,
        0.22652837, 1.        , 0.23639516, 0.18804414, 0.42011221],
       [0.27051885, 0.17826783, 0.2632513

# 2. Finding the optimal portfolio for different asset allocation models (83 marks)
Assume that the risk free asset has an expected return rf = 4.30%. Standard deviations and covariances should come from question 1 above, and the “expected 1-yr return” as provided in the spreadsheet should be used as the expected return of each stock.

## i. With Modern Portfolio Theory [58 marks]
a. Find the standard deviation and expected return of an equal weight (EW) portfolio, where all the stocks have a weight wi = 0.1.

In [5]:
# 导入 expected_return
expected_return = pd.read_csv('./data/expected_return.csv')
tmp = []
for e in expected_return['expected_return']:
    a = float(e.strip('%'))
    b = a/100
    e = round(b,6)
    tmp.append(e)
expected_return = np.array(tmp)
expected_return

array([0.089, 0.149, 0.121, 0.107, 0.22 , 0.065, 0.165, 0.104, 0.185,
       0.17 ])

In [6]:
# 风险无息资产的预期回报率为4.30%
rf = 0.043

w = 0.1

# 计算等权重组合的预期收益率
weights = np.array([w for i in range(len(expected_return))])
expected_portfolio_return = np.sum(expected_return * weights)
print(np.dot(annualized_std,annualized_std))
# 计算等权重组合的预期标准差
annualized_std = np.array(annualized_std)

# 计算等权重组合的预期标准差
portfolio_std = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

print("等权重组合的预期收益率：{:.2%}".format(expected_portfolio_return))
print("等权重组合的预期标准差：{:.2%}".format(portfolio_std))

1.4335398410933216
等权重组合的预期收益率：13.75%
等权重组合的预期标准差：1.41%
